# Strom-Endverbrauch Schweiz: Entwicklung nach Sektoren
Dieses Notebook analysiert und visualisiert den Strom-Endverbrauch in der Schweiz über die letzten Jahre, differenziert nach Sektoren.

## Datenimport, Aufbereitung & Anreicherung
In diesem Abschnitt werden die Rohdaten geladen, bereinigt und für die Visualisierung vorbereitet.

In [ ]:
# Bibliotheken importieren
import pandas as pd
import plotly.express as px

# CSV-Daten einlesen (Trennzeichen: Semikolon)
file_path = '../data/verbrauch/Strom_Endverbrauch_Schweiz.csv'
data = pd.read_csv(file_path, sep=';')

# Daten auf relevante Jahre beschränken (2015–2024)
data = data[(data['Jahr'] >= 2015) & (data['Jahr'] <= 2024)]

# Spaltennamen bereinigen: Entferne " in GWh" aus allen Spaltennamen
data.columns = [col.replace(' in GWh', '') for col in data.columns]

# Datenüberblick: Zeige die ersten 5 Zeilen
data.head()

## Interaktive Visualisierung: Stromverbrauch nach Sektor

In [ ]:
# Daten für Visualisierung transformieren: Wide- zu Long-Format
data_melted = data.melt(
    id_vars=['Jahr'],
    var_name='Sektor',
    value_name='Verbrauch (GWh)'
)

# Top-3 Sektoren nach Gesamtverbrauch bestimmen
top_sectors = (
    data_melted.groupby('Sektor')['Verbrauch (GWh)']
    .sum()
    .nlargest(3)
    .index
)

# Farbzuordnung: Top-3 Sektoren farbig, Rest grau
base_colors = ['#636efa', '#EF553B', '#00cc96']
color_map = {sector: color for sector, color in zip(top_sectors, base_colors)}
color_map.update({
    sector: 'lightgray'
    for sector in data_melted['Sektor'].unique()
    if sector not in top_sectors
})

# Liniendiagramm mit Plotly Express erstellen
fig = px.line(
    data_melted,
    x='Jahr',
    y='Verbrauch (GWh)',
    color='Sektor',
    title='Strom-Endverbrauch in der Schweiz nach Sektoren',
    labels={
        'Jahr': 'Jahr',
        'Verbrauch (GWh)': 'Verbrauch (GWh)',
        'Sektor': 'Sektor'
    },
    color_discrete_map=color_map
)

# Darstellung der Linien und Marker anpassen
fig.update_traces(
    mode='markers+lines',
    marker=dict(size=6),
    line=dict(width=2)
)

# Layout-Optimierungen: Achsen, Legende, Design
fig.update_layout(
    xaxis=dict(tickmode='linear', title='Jahr'),
    yaxis=dict(title='Verbrauch (GWh)'),
    legend_title='Sektor',
    template='simple_white',
    title=dict(font=dict(size=20))
)

# Diagramm als HTML-Datei speichern und anzeigen
fig.write_html('../docs/assets/diagramme/Strom_Endverbrauch_Schweiz.html')
fig.show()